In [1]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import os
import numpy as np

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/466\ Project/CSC466-Project

/content/drive/MyDrive/466 Project/CSC466-Project


In [3]:
df = pd.read_csv('total_data.csv')
df

,file type,file dist of bytes,consecutiveByte,consecutiveCount,hash,hash type,hash len,hash dist,hash 0 bits,hash 1 bits,hash len bits,file size (bytes)
0,.jpg,"[185, 71, 61, 64, 62, 43, 59, 58, 47, 49, 108,...",b'\n',50,fc02cfed7b327dfc8ea64daeb388803dda1f53df,SHA-1,40,"[2, 1, 2, 4, 1, 1, 1, 2, 4, 0, 3, 2, 3, 6, 3, 5]",69,91,160,14425
1,.jpg,"[192, 78, 55, 54, 53, 54, 61, 70, 50, 55, 115,...",b'\n',50,8a7712058d814572ff536384fff2dffa26169460d861b5...,SHA-256,64,"[4, 6, 5, 5, 4, 5, 6, 4, 6, 1, 4, 1, 1, 3, 2, 7]",133,123,256,13631
2,.jpg,"[181, 80, 57, 57, 63, 60, 63, 70, 44, 44, 115,...",b'\n',50,be27d2bb8b26fbd951851ac0b27fdd833ccf194d37da69...,SHA-3-256,64,"[2, 3, 5, 6, 2, 3, 3, 5, 3, 5, 3, 6, 5, 7, 1, 5]",116,140,256,16170
3,.jpg,"[215, 68, 51, 73, 52, 57, 52, 55, 40, 43, 102,...",b'\n',50,a56f71e3497a5bcf8b955dbe4bcb2497,MD5,32,"[0, 1, 1, 1, 3, 4, 1, 3, 1, 3, 2, 5, 2, 1, 2, 2]",55,73,128,14762
4,.jpg,"[191, 90, 60, 66, 56, 59, 66, 60, 53, 54, 102,...",b'\n',50,c772a6cc5373e3c06dfc359babd96775a91ab97498ecbe...,SHA-256,64,"[2, 2, 1, 4, 3, 4, 4, 9, 2, 7, 6, 5, 7, 3, 4, 1]",117,139,256,14517
...,...,...,...,...,...,...,...,...,...,...,...,...
5245,.pdf,"[10194, 3869, 3826, 3705, 3466, 3743, 3592, 38...",b' ',72,8b87f8aa3e04d37f66f64c3bff88b0efa0e9694f3833f4...,SHA-256,64,"[5, 2, 1, 9, 5, 1, 4, 2, 8, 2, 5, 5, 1, 2, 4, 8]",125,131,256,1039358
5246,.pdf,"[29855, 14763, 15433, 17860, 14182, 15818, 156...",b'\x00',359,461add4173b80d1781c438db98c5e4e200f0987847de9f...,SHA-256,64,"[4, 5, 3, 3, 6, 1, 2, 6, 7, 8, 1, 3, 3, 5, 5, 2]",134,122,256,4789771
5247,.pdf,"[3220, 3164, 3107, 3076, 3088, 3066, 3136, 319...",b' ',100,dcb2b9effcf8f1d92bb91ba8a3cd58516399b2431318eb...,SHA-256,64,"[1, 6, 4, 5, 2, 6, 2, 0, 5, 6, 3, 8, 3, 4, 4, 5]",121,135,256,1311284
5248,.pdf,"[1053, 896, 765, 768, 794, 783, 715, 760, 756,...",b'\x14',50,ae1ac4367178c45daef2878bf05e2487,MD5,32,"[1, 2, 2, 1, 3, 2, 1, 4, 4, 0, 3, 1, 2, 1, 3, 2]",64,64,128,226594


In [4]:
df['hash type'].value_counts()

SHA-1        1346
SHA-3-256    1338
SHA-256      1285
MD5          1281
Name: hash type, dtype: int64

The hashing algorithms are roughly equal in number, which is helpful since 

In [5]:
len(df), len(df['hash'].unique())

(5250, 5238)

Interestingly, there are some duplicate hashes.

In [6]:
df[df['hash'].duplicated(keep=False)]['hash']

4004             47ef54857f4d7f1ebdd98190085d57b329ca0401
4044    9b452145bdb3bee3d52a1b7c4a6003baab8c95a0cdf2a5...
4045    9b452145bdb3bee3d52a1b7c4a6003baab8c95a0cdf2a5...
4077    9ddac2408ffba1b6f2a1d5b40a99ac3aa7779ed058c478...
4078    9ddac2408ffba1b6f2a1d5b40a99ac3aa7779ed058c478...
4161             a7586d2bc67adf30a40fc1922ad544c3699b7c9e
4162             a7586d2bc67adf30a40fc1922ad544c3699b7c9e
4199             4afa6c2eb552ceef0e396fbfe449932492873034
4200             4afa6c2eb552ceef0e396fbfe449932492873034
4277                     4d11f40d372a2c181501e3b10b99b6b4
4278                     4d11f40d372a2c181501e3b10b99b6b4
4430    8783d73e0a8162a5360495349afda4a6f0a1b54ea2ebb8...
4442    2bd5e294df556e563aeaf511ec4a7a4676844da9de8959...
4443    2bd5e294df556e563aeaf511ec4a7a4676844da9de8959...
4448    0b7b7d91ec95add896ac5175dc2ed28fba86a515d70126...
4449    0b7b7d91ec95add896ac5175dc2ed28fba86a515d70126...
4476             8c2305e79c09ed1bf0a1dd6eaf79221f0d4784d0
4484          

In [7]:
df.loc[[5119,5120,3996,3997]]

,file type,file dist of bytes,consecutiveByte,consecutiveCount,hash,hash type,hash len,hash dist,hash 0 bits,hash 1 bits,hash len bits,file size (bytes)
5119,.pdf,"[34822, 3618, 3320, 3069, 3283, 3477, 3052, 39...",b'\x00',2051,81ca673bc5e7bb818fabe37e8bf9ac8e,MD5,32,"[0, 2, 0, 2, 0, 1, 1, 3, 5, 1, 3, 5, 3, 0, 4, 2]",55,73,128,993032
5120,.pdf,"[34822, 3618, 3320, 3069, 3283, 3477, 3052, 39...",b'\x00',2051,933d74f18eccba07ef91c768735ce4ad1d3fe1ab73b7b7...,SHA-256,64,"[2, 4, 1, 8, 2, 2, 5, 7, 2, 3, 5, 5, 5, 4, 4, 5]",111,145,256,993032
3996,.pdf,"[2436, 1301, 1178, 1162, 1214, 1118, 1241, 133...",b'\x00',220,a92a80aaabc2e52063b8233c47bf8e9b,MD5,32,"[2, 0, 4, 3, 1, 1, 1, 1, 3, 2, 5, 4, 2, 0, 2, 1]",67,61,128,430333
3997,.pdf,"[2436, 1301, 1178, 1162, 1214, 1118, 1241, 133...",b'\x00',220,20d95878f7b2f53dce900162e7b8959cb951e76ddd3e77...,SHA-3-256,64,"[6, 3, 3, 5, 1, 4, 4, 6, 4, 6, 0, 5, 4, 5, 5, 3]",124,132,256,430333


Seem to just be duplicated files; need to remove duplicates in data processing.

In [8]:
print(len(df))
df.drop_duplicates(ignore_index=True, inplace=True)
len(df)

5250


5238

Let's take a look at how the distribution of 0s and 1s in the bitwise representations of the hash varies based on the algorithm.

In [10]:
df.groupby('hash type').mean()[['hash 0 bits', 'hash 1 bits']]

,hash 0 bits,hash 1 bits
hash type,,
MD5,64.182812,63.817188
SHA-1,80.014169,79.985831
SHA-256,127.912568,128.087432
SHA-3-256,128.532934,127.467066


Note that this is due to the fact that some hashes are longer than other hashes. Let's see what happens if we normalize the bit counts.

In [11]:
df['0 bits norm'] = df['hash 0 bits'] / df['hash len bits']
df['1 bits norm'] = df['hash 1 bits'] / df['hash len bits']

df.groupby('hash type').mean()[['0 bits norm', '1 bits norm']]

,0 bits norm,1 bits norm
hash type,,
MD5,0.501428,0.498572
SHA-1,0.500089,0.499911
SHA-256,0.499658,0.500342
SHA-3-256,0.502082,0.497918


Once normalized, the 0 bits and 1 bits are around equally likekly (50% each) for all hashing algorithms.

However, it does make sense to exploit the length of a hash to determine the algorithm it was generated from.

In [12]:
df.groupby(['hash type', 'file type']).mean()[['0 bits norm', '1 bits norm']]

0 bits norm  1 bits norm
hash type file type                          
MD5       .jpg          0.496229     0.503771
          .json         0.506333     0.493667
          .pdf          0.503107     0.496893
          .txt          0.503010     0.496990
SHA-1     .jpg          0.499617     0.500383
          .json         0.500354     0.499646
          .pdf          0.503320     0.496680
          .txt          0.495692     0.504308
SHA-256   .jpg          0.499248     0.500752
          .json         0.501974     0.498026
          .pdf          0.499095     0.500905
          .txt          0.498505     0.501495
SHA-3-256 .jpg          0.501606     0.498394
          .json         0.499973     0.500027
          .pdf          0.503064     0.496936
          .txt          0.503736     0.496264

The same trend is exhibited across all pairs of hash and file types.

In [13]:
df['hash dist'] = df['hash dist'].apply(eval)

In [14]:
df_dist = pd.DataFrame(df['hash dist'].to_list(), 
                       index=df.index, 
                       columns=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f'])
df_dist['hash type'] = df['hash type']

df_dist

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f,hash type
0,2,1,2,4,1,1,1,2,4,0,3,2,3,6,3,5,SHA-1
1,4,6,5,5,4,5,6,4,6,1,4,1,1,3,2,7,SHA-256
2,2,3,5,6,2,3,3,5,3,5,3,6,5,7,1,5,SHA-3-256
3,0,1,1,1,3,4,1,3,1,3,2,5,2,1,2,2,MD5
4,2,2,1,4,3,4,4,9,2,7,6,5,7,3,4,1,SHA-256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5233,5,2,1,9,5,1,4,2,8,2,5,5,1,2,4,8,SHA-256
5234,4,5,3,3,6,1,2,6,7,8,1,3,3,5,5,2,SHA-256
5235,1,6,4,5,2,6,2,0,5,6,3,8,3,4,4,5,SHA-256
5236,1,2,2,1,3,2,1,4,4,0,3,1,2,1,3,2,MD5


In [15]:
df_dist.groupby('hash type').mean()

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
hash type,,,,,,,,,,,,,,,,
MD5,2.025000,2.014844,2.031250,1.975000,1.967969,1.957031,2.003125,2.021875,2.032031,1.974219,2.031250,1.969531,2.001562,2.078125,1.990625,1.926562
SHA-1,2.468307,2.507084,2.502610,2.488441,2.410142,2.536167,2.477256,2.544370,2.551827,2.473527,2.539896,2.479493,2.592841,2.539150,2.446682,2.442207
SHA-256,4.001561,3.932084,4.062451,3.939891,3.960968,4.028884,3.993755,3.982826,4.032787,3.972678,3.992194,4.099141,3.987510,4.032787,3.996877,3.983607
SHA-3-256,4.057635,4.029940,4.059880,4.107784,3.985778,3.987275,4.053892,3.962575,4.047904,3.997754,3.978293,3.976796,4.004491,3.880240,3.853293,4.016467


The above data, once again, is based on non-normalized counts of the occurrence of each hash value. However, we see some minor (albeit maybe significant) values - SHA-256 and SHA-3-256 have differences of up to 4% in some byte frequency averages (byte e). Let's see the normalized output:

In [16]:
df_dist2 = df_dist.drop('hash type', axis=1)

df_dist2 = df_dist2.div(df_dist2.sum(axis=1), axis=0)
df_dist2['hash type'] = df['hash type']
df_dist2.head()

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f,hash type
0,0.05000,0.025000,0.050000,0.100000,0.025000,0.025000,0.025000,0.050000,0.100000,0.000000,0.075000,0.050000,0.075000,0.150000,0.075000,0.125000,SHA-1
1,0.06250,0.093750,0.078125,0.078125,0.062500,0.078125,0.093750,0.062500,0.093750,0.015625,0.062500,0.015625,0.015625,0.046875,0.031250,0.109375,SHA-256
2,0.03125,0.046875,0.078125,0.093750,0.031250,0.046875,0.046875,0.078125,0.046875,0.078125,0.046875,0.093750,0.078125,0.109375,0.015625,0.078125,SHA-3-256
3,0.00000,0.031250,0.031250,0.031250,0.093750,0.125000,0.031250,0.093750,0.031250,0.093750,0.062500,0.156250,0.062500,0.031250,0.062500,0.062500,MD5
4,0.03125,0.031250,0.015625,0.062500,0.046875,0.062500,0.062500,0.140625,0.031250,0.109375,0.093750,0.078125,0.109375,0.046875,0.062500,0.015625,SHA-256


In [17]:
hash_norm_dists = df_dist2.groupby('hash type').mean()
hash_norm_dists

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
hash type,,,,,,,,,,,,,,,,
MD5,0.063281,0.062964,0.063477,0.061719,0.061499,0.061157,0.062598,0.063184,0.063501,0.061694,0.063477,0.061548,0.062549,0.064941,0.062207,0.060205
SHA-1,0.061708,0.062677,0.062565,0.062211,0.060254,0.063404,0.061931,0.063609,0.063796,0.061838,0.063497,0.061987,0.064821,0.063479,0.061167,0.061055
SHA-256,0.062524,0.061439,0.063476,0.061561,0.061890,0.062951,0.062402,0.062232,0.063012,0.062073,0.062378,0.064049,0.062305,0.063012,0.062451,0.062244
SHA-3-256,0.063401,0.062968,0.063436,0.064184,0.062278,0.062301,0.063342,0.061915,0.063249,0.062465,0.062161,0.062137,0.062570,0.060629,0.060208,0.062757


In [18]:
expected_rel_freq = 1/16
abs(hash_norm_dists.iloc[2] - hash_norm_dists.iloc[3]) / expected_rel_freq

0    0.014018
1    0.024464
2    0.000643
3    0.041973
4    0.006203
5    0.010402
6    0.015034
7    0.005063
8    0.003779
9    0.006269
a    0.003475
b    0.030586
c    0.004245
d    0.038137
e    0.035896
f    0.008215
dtype: float64

As noted before, some of the differences are appreciably large, whereas the others aren't. For example, hex values d, e, and 3 all have around a 4% difference compared to the expected value. That is 2 orders of magnitude greater than the 0.06% difference in byte 2.

Therefore, it seems like the distribution of bytes might be an important feature to include while training models.

Note that this notebook has only covered basic (exploratory) analysis, and we haven't even considered properties of the file/string the hash was generated from, nor have we considered interactions between those and the hash properties we have here. Perhaps some combination of the available features will lead to a successful model capable of identifying the algorithm.

In [70]:
df_chunk = df[['file type', 'hash', 'hash type']]
df_chunk

chunk1 = []
chunk2 = []
chunk3 = []
chunk1_1_bits = []
chunk2_1_bits = []
chunk3_1_bits = []

for i in range(len(df_chunk)):
  curr_hash = df_chunk.loc[i,'hash']
  chunk_len = len(curr_hash) // 3
  chunk1.append(curr_hash[:chunk_len])
  chunk2.append(curr_hash[chunk_len:2*chunk_len])
  chunk3.append(curr_hash[2*chunk_len:])

  chunk1_1_bits.append(bin(int(chunk1[i], 16))[2:].count('1'))
  chunk2_1_bits.append(bin(int(chunk2[i], 16))[2:].count('1'))
  chunk3_1_bits.append(bin(int(chunk3[i], 16))[2:].count('1'))

df_chunk.loc[:, 'chunk1'] = pd.Series(chunk1)
df_chunk.loc[:, 'chunk2'] = pd.Series(chunk2)
df_chunk.loc[:, 'chunk3'] = pd.Series(chunk3)
df_chunk['chunk1_num_bits'] = 4 * df_chunk['chunk1'].str.len()
df_chunk['chunk2_num_bits'] = 4 * df_chunk['chunk2'].str.len()
df_chunk['chunk3_num_bits'] = 4 * df_chunk['chunk3'].str.len()

df_chunk.loc[:, 'chunk1_1_bits'] = pd.Series(chunk1_1_bits)
df_chunk.loc[:, 'chunk2_1_bits'] = pd.Series(chunk2_1_bits)
df_chunk.loc[:, 'chunk3_1_bits'] = pd.Series(chunk3_1_bits)
df_chunk['chunk1_0_bits'] = df_chunk['chunk1_num_bits'] - df_chunk['chunk1_1_bits']
df_chunk['chunk2_0_bits'] = df_chunk['chunk2_num_bits'] - df_chunk['chunk2_1_bits']
df_chunk['chunk3_0_bits'] = df_chunk['chunk3_num_bits'] - df_chunk['chunk3_1_bits']

df_chunk

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,file type,hash,hash type,chunk1,chunk2,chunk3,chunk1_num_bits,chunk2_num_bits,chunk3_num_bits,chunk1_1_bits,chunk2_1_bits,chunk3_1_bits,chunk1_0_bits,chunk2_0_bits,chunk3_0_bits
0,.jpg,fc02cfed7b327dfc8ea64daeb388803dda1f53df,SHA-1,fc02cfed7b327,dfc8ea64daeb3,88803dda1f53df,52,52,56,31,31,29,21,21,27
1,.jpg,8a7712058d814572ff536384fff2dffa26169460d861b5...,SHA-256,8a7712058d814572ff536,384fff2dffa26169460d8,61b5c7300e18e5aa342613,84,84,88,40,45,38,44,39,50
2,.jpg,be27d2bb8b26fbd951851ac0b27fdd833ccf194d37da69...,SHA-3-256,be27d2bb8b26fbd951851,ac0b27fdd833ccf194d37,da693f296c734039da5fc7,84,84,88,46,46,48,38,38,40
3,.jpg,a56f71e3497a5bcf8b955dbe4bcb2497,MD5,a56f71e349,7a5bcf8b95,5dbe4bcb2497,40,40,48,22,24,27,18,16,21
4,.jpg,c772a6cc5373e3c06dfc359babd96775a91ab97498ecbe...,SHA-256,c772a6cc5373e3c06dfc3,59babd96775a91ab97498,ecbe0778ba15497d9c4a6e,84,84,88,46,46,47,38,38,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5233,.pdf,8b87f8aa3e04d37f66f64c3bff88b0efa0e9694f3833f4...,SHA-256,8b87f8aa3e04d37f66f64,c3bff88b0efa0e9694f38,33f41a332b04a01d8e3b85,84,84,88,46,46,39,38,38,49
5234,.pdf,461add4173b80d1781c438db98c5e4e200f0987847de9f...,SHA-256,461add4173b80d1781c43,8db98c5e4e200f0987847,de9fb99327c27968e9e941,84,84,88,37,37,48,47,47,40
5235,.pdf,dcb2b9effcf8f1d92bb91ba8a3cd58516399b2431318eb...,SHA-256,dcb2b9effcf8f1d92bb91,ba8a3cd58516399b24313,18eb564815a59f5ebe203d,84,84,88,52,39,44,32,45,44
5236,.pdf,ae1ac4367178c45daef2878bf05e2487,MD5,ae1ac43671,78c45daef2,878bf05e2487,40,40,48,19,22,23,21,18,25


In [72]:
df_chunk.groupby('hash type').mean()[['chunk1_0_bits', 'chunk1_1_bits']]

,chunk1_1_bits,chunk1_0_bits
hash type,,
MD5,20.010156,19.989844
SHA-1,25.994780,26.005220
SHA-256,41.939110,42.060890
SHA-3-256,41.793413,42.206587


In [73]:
df_chunk.groupby('hash type').mean()[['chunk2_0_bits', 'chunk2_1_bits']]

,chunk2_0_bits,chunk2_1_bits
hash type,,
MD5,19.950781,20.049219
SHA-1,25.841909,26.158091
SHA-256,41.871975,42.128025
SHA-3-256,42.205838,41.794162


In [74]:
df_chunk.groupby('hash type').mean()[['chunk3_0_bits', 'chunk3_1_bits']]

,chunk3_0_bits,chunk3_1_bits
hash type,,
MD5,24.242188,23.757812
SHA-1,28.167040,27.832960
SHA-256,43.979703,44.020297
SHA-3-256,44.120509,43.879491


Despite the chunking, we seem to see no significant diffrence (variance seems to be staying constant) when chunking the hash and then calculating values from it.

# TO DO:
Split hashess into chunks of 3,5,10 etc and run the same analyses; any differences observed?